In [1]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import time


In [ ]:

options = webdriver.ChromeOptions()
# options.add_argument()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")

# Criar uma instância única do ChromeDriver
driver = webdriver.Chrome(options=options)
driver.maximize_window()

# Lista para armazenar todas as URLs dos anúncios
urls = []

for i in range(1, 31):  # Busca nas primeiras 30 páginas
    url = f"https://www.zapimoveis.com.br/venda/casas/?transacao=venda&tipos=casa_residencial&pagina={i}"
    driver.get(url)
    time.sleep(3)  # Espera para garantir que a página carregue

    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Pegar o HTML atualizado e extrair os links
    soup = BeautifulSoup(driver.page_source, "html.parser")
    anuncios = soup.find_all("a", {"itemtype": "https://schema.org/House"})

    for anuncio in anuncios:
        link = anuncio["href"]
        if not link.startswith("http"):
            link = "https://www.zapimoveis.com.br" + link  # Corrigir links relativos
        urls.append(link)

# Fechar o navegador Selenium
driver.quit()

# Exibir os links coletados
for link in urls:
    print(link)

TypeError: ArgOptions.add_argument() missing 1 required positional argument: 'argument'

In [112]:
print(len(urls))

69


In [ ]:
df = pd.DataFrame(columns=["Link", "Área", "Banheiro", "Estacionamento", "Suíte", "Quartos", "Valor Imóvel", "Descrição", "Endereço", "Acabamento","Tipo Imovel"])

# Função para extrair acabamentos da descrição
def extrair_acabamentos(texto):
    padrao = r"porcelanato|ceramica|cerâmica|cerâmico|cerâmicos|gesso|antiderrapante|planejado|tábua"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None
scraper = cloudscraper.create_scraper()

# Função para extrair o tipo de imóvel da descrição
def extrair_tipo_imovel(texto):
    padrao = r"casa|apartamento|sobrado|fazenda"
    if isinstance(texto, str):
        encontrados = re.findall(padrao, texto.lower())
        return ", ".join(set(encontrados)) if encontrados else None
    return None
scraper = cloudscraper.create_scraper()

# Loop para acessar cada anúncio
for url in urls:
    
    response = scraper.get(url, headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        # Criar um dicionário com os dados do imóvel
        dados_imovel = {
            "Link": url,
            "Área": soup.find("li", {"itemprop": "floorSize"}).get_text(strip=True).split(" ")[0] if soup.find("li", {"itemprop": "floorSize"}) else None,
            "Banheiro": soup.find("li", {"itemprop": "numberOfBathroomsTotal"}).get_text(strip=True).split(" ")[0] if soup.find("li", {"itemprop": "numberOfBathroomsTotal"}) else None,
            "Estacionamento": soup.find("li", {"itemprop": "numberOfParkingSpaces"}).get_text(strip=True).split(" ")[0] if soup.find("li", {"itemprop": "numberOfParkingSpaces"}) else None,
            "Suíte": soup.find("li", {"itemprop": "numberOfSuites"}).get_text(strip=True).split(" ")[0] if soup.find("li", {"itemprop": "numberOfSuites"}) else None,
            "Quartos": soup.find("li", {"itemprop": "numberOfRooms"}).get_text(strip=True).split(" ")[0] if soup.find("li", {"itemprop": "numberOfRooms"}) else None,
            "Valor Imóvel": soup.find("p", {"data-testid": "price-info-value"}).get_text(strip=True) if soup.find("p", {"data-testid": "price-info-value"}) else None,
            "Descrição": soup.find("p", {"data-testid": "description-content"}).get_text(strip=True) if soup.find("p", {"data-testid": "description-content"}) else None,
            "Endereço": soup.find("p", {"data-testid": "address-info-value"}).get_text(strip=True) if soup.find("p", {"data-testid": "address-info-value"}) else None,
        }

        # Adicionar o acabamento extraído da Descrição
        dados_imovel["Acabamento"] = extrair_acabamentos(dados_imovel["Descrição"])
        
        # Adicionar o tipo de imóvel extraído da Descrição
        dados_imovel["Tipo Imovel"] = extrair_tipo_imovel(dados_imovel["Descrição"])

        # Adicionar os dados ao DataFrame
        df.loc[len(df)] = dados_imovel
    else:
        print(f"Erro ao acessar o anúncio: {url} - Código {response.status_code}")

# Exibir os dados coletados
df

,Link,Área,Banheiro,Estacionamento,Suíte,Quartos,Valor Imóvel,Descrição,Endereço,Acabamento,Tipo Imovel
0,https://www.zapimoveis.com.br/imovel/venda-cas...,1300,4,10,4,4,R$ 19.800.000,"Casa contemporânea Condomínio Mansões VAZIA, 1...","Rua Benjamim Pesset - Barra da Tijuca, Rio de ...",tábua,casa
1,https://www.zapimoveis.com.br/imovel/venda-cas...,364,6,4,4,4,R$ 3.500.000,Sofisticação e conforto no Alphaville Nova Esp...,"Rua Júlio Cassola, 379 - Alphaville Nova Espla...",None,None
2,https://www.zapimoveis.com.br/imovel/venda-cas...,150,3,4,1,3,R$ 770.000,"Linda casa linear, ótima localização a poucos ...","Rua dos Apóstolos - Manguinhos, Serra - ES",None,casa
3,https://www.zapimoveis.com.br/imovel/venda-cas...,80,2,2,1,3,R$ 280.000,Vende-se Linda casa Quiririm em TaubatéAcabame...,"Rua Doutor Luis André Gadioli, 57 - Quiririm, ...",None,casa
4,https://www.zapimoveis.com.br/imovel/venda-sob...,170,3,None,1,2,R$ 340.000,Vendo casa inacabada no bairro Estoril em Taub...,Rua Fernando Silveira Queiroz - Residencial Es...,None,casa
...,...,...,...,...,...,...,...,...,...,...,...
64,https://www.zapimoveis.com.br/imovel/venda-cas...,150,1,2,1,3,R$ 390.000,"Casa para Venda no bairro Savoy, localizado na...","Savoy , Itanhaém - SP",None,casa
65,https://www.zapimoveis.com.br/imovel/venda-cas...,300,3,5,None,3,R$ 380.000,"Venda de Casa 3 Dormitórios Santa Julia, Itanh...","Balneario Santa Julia, Itanhaém - SP",cerâmica,casa
66,https://www.zapimoveis.com.br/imovel/venda-cas...,125,1,2,1,2,R$ 229.999,"Casa para Venda no bairro Bopiranga, localizad...","Bopiranga, Itanhaém - SP",None,casa
67,https://www.zapimoveis.com.br/imovel/venda-sob...,144,1,2,1,3,R$ 420.000,Vende-se Sobrado na Granja Daniel em TaubatéLi...,"Rua Marcos Ferreira de Andrade Junqueira, 111 ...",None,sobrado


In [ ]:
df.to_excel("imoveis.xlsx", index=False)